In [1]:
import pandas as pd
import numpy as np

#读取文件
sojournTime_raw = pd.read_csv('sojournTime.csv',index_col=0)
comDelay_raw    = pd.read_csv('comDelay.csv',index_col=0)
load_raw     = pd.read_csv('load.csv',index_col=0)
memoryUtil_raw  = pd.read_csv('memoryUtil.csv',index_col=0)

# 得到cloudlet数量
cloudlet_numbers = sojournTime_raw.shape[1]

# 对comDelay和load进行同向化
comDelay_txh = comDelay_raw.apply(lambda x: (np.max(x) - x)) 
load_txh = load_raw.apply(lambda x: (np.max(x) - x)) 

# 合并成一个DataFrame
data = pd.concat([sojournTime_raw,comDelay_txh,load_txh,memoryUtil_raw],axis=1)

# 对所有数据进行归一化
data_nor = data.apply(lambda x: (x - np.min(x)) / (np.max(x) - np.min(x)))
data_nor


,sojournTime0,sojournTime1,sojournTime2,sojournTime3,sojournTime4,sojournTime5,sojournTime6,sojournTime7,sojournTime8,sojournTime9,...,memoryUtil10,memoryUtil11,memoryUtil12,memoryUtil13,memoryUtil14,memoryUtil15,memoryUtil16,memoryUtil17,memoryUtil18,memoryUtil19
T0,0.832740,0.586957,0.460121,0.000000,0.652570,0.323390,0.476558,0.548098,0.684534,1.000000,...,0.538462,1.000000,0.137931,0.027778,0.184211,0.000000,0.350,0.717949,0.878788,0.631579
T1,0.763701,0.000000,0.522002,1.000000,0.646373,1.000000,0.989204,0.321402,0.707147,0.679563,...,0.666667,0.230769,0.827586,0.861111,0.447368,0.470588,0.875,1.000000,1.000000,0.000000
T2,1.000000,0.918116,0.825908,0.128447,1.000000,0.578470,0.391117,0.439722,0.593802,0.658399,...,0.205128,0.333333,0.655172,0.138889,1.000000,0.823529,0.000,0.256410,0.090909,0.078947
T3,0.459786,0.518116,0.000000,0.245724,0.587678,0.767834,0.729180,0.700224,0.000000,0.496693,...,0.589744,0.974359,0.379310,0.555556,0.052632,0.764706,0.100,0.794872,0.696970,1.000000
T4,0.665480,0.802899,0.655941,0.005236,0.127598,0.000000,0.000000,0.393985,0.593802,0.853505,...,0.128205,0.179487,0.000000,0.138889,0.236842,0.264706,0.675,0.538462,0.636364,0.947368
T5,0.279715,0.408696,1.000000,0.198953,0.471382,0.040208,0.878470,0.000000,0.414015,0.325066,...,0.000000,0.820513,0.586207,1.000000,0.631579,0.117647,1.000,0.820513,0.878788,0.552632
T6,0.565125,0.200725,0.061881,0.446771,0.825738,0.377432,1.000000,0.626895,0.649637,0.000000,...,0.897436,0.282051,0.068966,0.000000,0.052632,0.352941,0.400,0.717949,0.000000,0.500000
T7,0.000000,0.444203,0.817932,0.309948,0.114473,0.664073,0.121530,1.000000,1.000000,0.884590,...,0.564103,0.846154,0.413793,0.000000,0.657895,1.000000,0.725,0.512821,0.030303,0.184211
T8,0.348754,0.432609,0.244224,0.761257,0.000000,0.594466,0.773288,0.559781,0.790899,0.539352,...,0.692308,0.025641,0.068966,0.694444,0.000000,0.470588,0.125,0.000000,0.303030,0.710526
T9,0.772242,1.000000,0.561606,0.919721,0.075100,0.894942,0.232572,0.785235,0.536013,0.633929,...,1.000000,0.000000,1.000000,0.583333,0.315789,0.764706,0.525,0.102564,0.515152,0.236842


In [2]:
# 云模型处理
def cloud_model(df):
    Ex = df.mean()
    En = np.sqrt(np.pi/2)  * np.mean( np.abs(df-Ex) )
    He = np.sqrt(np.abs(df.mean() * df.mean() - df.std() * df.std()))
    return pd.concat([Ex,En,He],axis=1)
data_cloudModel = cloud_model(data_nor)
data_cloudModel

,0,1,2
sojournTime0,0.568754,0.298387,0.482999
sojournTime1,0.531232,0.296545,0.430711
sojournTime2,0.514961,0.324262,0.393567
sojournTime3,0.401606,0.381340,0.158778
sojournTime4,0.450091,0.371781,0.283035
...,...,...,...
memoryUtil15,0.502941,0.336183,0.381067
memoryUtil16,0.477500,0.354061,0.333582
memoryUtil17,0.546154,0.331003,0.434900
memoryUtil18,0.503030,0.398022,0.334348


In [3]:
# 中智聚合

# 中智数
truth = data_cloudModel[0]
indeterminacy = data_cloudModel[1]
falsity = data_cloudModel[2]

# 权重
weight = [0.25,0.25,0.25,0.25]

# 聚合之后的中智数
aggre_truth = []
aggre_indeterminacy = []
aggre_falsity = []

for i in range(cloudlet_numbers):
    tempT = 1
    tempI = 1
    tempF = 1
    # 四个属性
    for j in range(4):
        tempT *= (1 - truth[i+10*j]) ** weight[j]
        tempI *= indeterminacy[i+10*j] ** weight[j]
        tempF *= falsity[i+10*j] ** weight[j]
    aggre_truth.append((1-tempT))
    aggre_indeterminacy.append(tempI)
    aggre_falsity.append(tempF)

# 添加索引
# index = pd.Series(np.arange(0,cloudlet_numbers))
# index = index.astype(str)
# index = 'cloudlet'+index
neutrosophic = pd.DataFrame(np.vstack((aggre_truth,aggre_indeterminacy,aggre_falsity)).T)
neutrosophic


,0,1,2
0,0.489918,0.322589,0.354315
1,0.494205,0.298039,0.369461
2,0.480779,0.331620,0.347775
3,0.427880,0.404910,0.178849
4,0.511389,0.351348,0.366663
5,0.511480,0.330457,0.299266
6,0.534875,0.365962,0.371618
7,0.469619,0.308636,0.314352
8,0.528015,0.301241,0.399701
9,0.486229,0.345530,0.300278


In [4]:
# 得分函数
score = neutrosophic[0]+1-neutrosophic[1]+1-neutrosophic[2]
print(score)
print('最佳cloudlet为：',score.idxmax())

with open("optimal_cloudlet.txt","w") as f:
        f.write(str(score.idxmin()))  #这句话自带文件关闭功能，不需要再写f.close()


0     1.813014
1     1.826705
2     1.801383
3     1.844120
4     1.793377
5     1.881758
6     1.797295
7     1.846631
8     1.827072
9     1.840421
10    1.817063
11    1.829646
12    1.827459
13    1.867218
14    1.791103
15    1.886327
16    1.827165
17    1.838243
18    1.797536
19    1.798924
dtype: float64
最佳cloudlet为： 15
